#### Prepare notebook with relevant modules

In [1]:
!pip install -U odp-sdk --quiet

DEPRECATION: Loading egg at /srv/conda/envs/notebook/lib/python3.11/site-packages/odp-0.0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.17.56 requires cryptography<40.0.2,>=40.0.0, but you have cryptography 44.0.1 which is incompatible.
pyopenssl 23.2.0 requires cryptography!=40.0.0,!=40.0.1,<42,>=38.0.0, but you have cryptography 44.0.1 which is incompatible.


In [30]:
from odp.client import OdpClient # The SDK
from shapely.geometry import box
import pandas as pd
import pydeck as pdk

In [31]:
def create_h3_pydeck_map(df, hex_column, value_column, scale_factor=500, center_lat=58.0, center_lon=10.0, zoom=5):
    return pdk.Deck(
        layers=[pdk.Layer( 'H3HexagonLayer', df, get_hexagon=hex_column, get_fill_color=f'[254, 119, 76, {scale_factor} * {value_column}]', opacity=1, extruded=False,
            get_line_color=[32, 10, 58, 10], line_width_min_pixels=1, pickable=True, filled=True, stroked=True )],
        initial_view_state=pdk.ViewState(latitude=center_lat, longitude=center_lon, zoom=zoom, pitch=0), map_provider="mapbox", map_style='mapbox://styles/oceandatafoundation/clwg6xklg00an01pcgmeufjxq', api_keys={'mapbox': 'pk.eyJ1Ijoib2NlYW5kYXRhZm91bmRhdGlvbiIsImEiOiJjazk5bGxpNWkwYWU1M2Vya3hkcHh4czdrIn0.yf7kIiPfDNE7KP9_9wTN6A'})

In [32]:
base_url = "https://odcat.dev.hubocean.io"
client = OdpClient(base_url=base_url)

In [33]:
## Request the dataset from the catalog using the UUID:
osa_table_dataset = client.catalog.get(("7af5bc0f-c12a-451b-9834-fa6c4e0a2c87"))
osa_data = client.table_v2(osa_table_dataset)
osa_table_dataset.metadata.display_name

'Ocean Sensitive Areas - version 1.0.0 - table'

In [41]:
osa_table_dataset.spec.facets

{'Lineage': ['https://app.hubocean.earth/catalog/collection/moura2016-amazoncoral',
  'https://app.hubocean.earth/catalog/collection/allencoralatlas',
  'https://app.hubocean.earth/catalog/collection/gmw',
  'https://app.hubocean.earth/catalog/collection/seamounts2023',
  'https://app.hubocean.earth/catalog/collection/1e3401d4-9630-40cd-a9cf-d875cb310449-world-ocean-seamounts',
  'https://app.hubocean.earth/catalog/dataset/obis-metadata',
  'https://app.hubocean.earth/catalog/dataset/wcmc-coral',
  'https://app.hubocean.earth/catalog/dataset/wcmc-seagrass',
  'https://app.hubocean.earth/catalog/dataset/copernicus-global-physics-monthly'],
 'Limitations': 'This is a sample of the Ocean Sensitive Areas data product, which is to be used for only research and investigation purposes, not to be used for any commercial or decision-making purposes without written permission from HUB Ocean'}

In [20]:
#Get sample of the full data table
next(osa_data.select().dataframes()).head(3)

,shannon,simpson,mangrove,hex6,seamount,cold_water_coral,seagrass,geometry,coral,S,shannon_norm,S_raw
0,3.689,0.025,0.0,864f45d4fffffff,0.0,0.0,0.0,"POLYGON ((148.9515 12.038, 148.9448 12.0033, 1...",0.0,8.6,0.461,0.077
1,0.000,0.000,0.0,864f61647ffffff,1.0,0.0,0.0,"POLYGON ((148.6306 10.6888, 148.6239 10.6537, ...",0.0,17.5,0.000,0.167
2,0.000,0.000,0.0,864f6164fffffff,1.0,0.0,0.0,"POLYGON ((148.5692 10.7014, 148.5626 10.6664, ...",0.0,17.5,0.000,0.167


In [10]:
# Bounding box for Southern Norway
# Enter min and max latitude and longitude values to create a bounding box polygon below. Or use the structure below to add any Well-Known-Text or GeoJSON defined polygon.
lat_min = 56
lat_max = 62
lon_min = 8
lon_max = 12
query_geometry = box(lon_min, lat_min, lon_max, lat_max).wkt
query_geometry

'POLYGON ((12 56, 12 62, 8 62, 8 56, 12 56))'

In [25]:
# Bounding box for Caribbean
# Enter min and max latitude and longitude values to create a bounding box polygon below. Or use the structure below to add any Well-Known-Text or GeoJSON defined polygon.
lat_min = 18
lat_max = 21
lon_min = -77
lon_max = -72
query_geometry = box(lon_min, lat_min, lon_max, lat_max).wkt
query_geometry

'POLYGON ((-72 18, -72 21, -77 21, -77 18, -72 18))'

In [26]:
# Request all data in the query_geometry polygon and format as a dataframe
df = pd.concat(list(osa_data.select(f'geometry within "{query_geometry}"').dataframes()), ignore_index=True).drop(columns=['geometry'])

In [27]:
df.tail(2)

,shannon,simpson,mangrove,hex6,seamount,cold_water_coral,seagrass,coral,S,shannon_norm,S_raw
1524,0.0,0.0,0.0,866736d6fffffff,1.0,0.0,0.0,0.0,17.5,0.0,0.167
1525,0.0,0.0,0.0,866736d77ffffff,1.0,0.0,0.0,0.0,17.5,0.0,0.167


In [29]:
# Vizualize data sample for the Shannon index. You can swap out shannon_norm for any other attribute e.g. seagrass
map_viz = create_h3_pydeck_map(df, 'hex6', 'shannon_norm', scale_factor=500, center_lat=((lat_min + lat_max) / 2), center_lon=((lon_min + lon_max) / 2), zoom=6)
map_viz

{
  "initialViewState": {
    "latitude": 19.5,
    "longitude": -74.5,
    "pitch": 0,
    "zoom": 6
  },
  "layers": [
    {
      "@@type": "H3HexagonLayer",
      "data": [
        {
          "S": 1.0,
          "S_raw": 0.0,
          "cold_water_coral": 0.0,
          "coral": 0.0,
          "hex6": "864590537ffffff",
          "mangrove": 0.0,
          "seagrass": 0.0,
          "seamount": 0.0,
          "shannon": 0.0,
          "shannon_norm": 0.0,
          "simpson": 1.0
        },
        {
          "S": 1.0,
          "S_raw": 0.0,
          "cold_water_coral": 0.0,
          "coral": 0.0,
          "hex6": "86459061fffffff",
          "mangrove": 0.0,
          "seagrass": 0.0,
          "seamount": 0.0,
          "shannon": 0.0,
          "shannon_norm": 0.0,
          "simpson": 1.0
        },
        {
          "S": 34.0,
          "S_raw": 0.333,
          "cold_water_coral": 0.0,
          "coral": 1.0,
          "hex6": "864590c87ffffff",
          "mangrove": 0.0,
          "seagrass": 1.0,
          "seamount": 0.0,
          "shannon": 0.0,
          "shannon_norm": 0.0,
          "simpson": 0.0
        },
        {
          "S": 34.0,
          "S_raw": 0.333,
          "cold_water_coral": 0.0,
          "coral": 1.0,
          "hex6": "864590c9fffffff",
          "mangrove": 0.0,
          "seagrass": 1.0,
          "seamount": 0.0,
          "shannon": 0.0,
          "shannon_norm": 0.0,
          "simpson": 0.0
        },
        {
          "S": 34.0,
          "S_raw": 0.333,
          "cold_water_coral": 0.0,
          "coral": 1.0,
          "hex6": "864590cafffffff",
          "mangrove": 0.0,
          "seagrass": 1.0,
          "seamount": 0.0,
          "shannon": 0.0,
          "shannon_norm": 0.0,
          "simpson": 0.0
        },
        {
          "S": 3.264,
          "S_raw": 0.023,
          "cold_water_coral": 0.0,
          "coral": 0.0,
          "hex6": "864590cf7ffffff",
          "mangrove": 0.0,
          "seagrass": 0.0,
          "seamount": 0.0,
          "shannon": 1.099,
          "shannon_norm": 0.137,
          "simpson": 0.333
        },
        {
          "S": 4.927,
          "S_raw": 0.04,
          "cold_water_coral": 0.0,
          "coral": 0.0,
          "hex6": "864592547ffffff",
          "mangrove": 0.0,
          "seagrass": 0.0,
          "seamount": 0.0,
          "shannon": 1.906,
          "shannon_norm": 0.238,
          "simpson": 0.156
        },
        {
          "S": 1.0,
          "S_raw": 0.0,
          "cold_water_coral": 0.0,
          "coral": 0.0,
          "hex6": "864592667ffffff",
          "mangrove": 0.0,
          "seagrass": 0.0,
          "seamount": 0.0,
          "shannon": 0.0,
          "shannon_norm": 0.0,
          "simpson": 1.0
        },
        {
          "S": 17.5,
          "S_raw": 0.167,
          "cold_water_coral": 0.0,
          "coral": 0.0,
          "hex6": "8645926b7ffffff",
          "mangrove": 1.0,
          "seagrass": 0.0,
          "seamount": 0.0,
          "shannon": 0.0,
          "shannon_norm": 0.0,
          "simpson": 0.0
        },
        {
          "S": 50.5,
          "S_raw": 0.5,
          "cold_water_coral": 0.0,
          "coral": 1.0,
          "hex6": "864592927ffffff",
          "mangrove": 1.0,
          "seagrass": 1.0,
          "seamount": 0.0,
          "shannon": 0.0,
          "shannon_norm": 0.0,
          "simpson": 0.0
        },
        {
          "S": 34.0,
          "S_raw": 0.333,
          "cold_water_coral": 0.0,
          "coral": 1.0,
          "hex6": "864592937ffffff",
          "mangrove": 0.0,
          "seagrass": 1.0,
          "seamount": 0.0,
          "shannon": 0.0,
          "shannon_norm": 0.0,
          "simpson": 0.0
        },
        {
          "S": 50.5,
          "S_raw": 0.5,
          "cold_water_coral": 0.0,
          "coral": 1.0,
          "hex6": "864592d07ffffff",
          "mangrove": 1.0,
          "seagrass": 1.0,
          "seamount

In [ ]:
### Select data from a single point

In [49]:
list(osa_data.select(f'geometry contains "POINT(128.38 0.39)"').rows())

[{'shannon': 0.0,
  'simpson': 0.0,
  'mangrove': 1.0,
  'hex6': '867393917ffffff',
  'seamount': 0.0,
  'cold_water_coral': 0.0,
  'seagrass': 1.0,
  'geometry': 'POLYGON ((128.349 0.4004, 128.3434 0.3648, 128.3711 0.3417, 128.4045 0.3542, 128.4101 0.3898, 128.3824 0.4129, 128.349 0.4004))',
  'coral': 1.0,
  'S': 50.5,
  'shannon_norm': 0.0,
  'S_raw': 0.5}]

In [50]:
next(osa_data.select('geometry contains "POINT(128.38 0.39)"').dataframes())

,shannon,simpson,mangrove,hex6,seamount,cold_water_coral,seagrass,geometry,coral,S,shannon_norm,S_raw
0,0.0,0.0,1.0,867393917ffffff,0.0,0.0,1.0,"POLYGON ((128.349 0.4004, 128.3434 0.3648, 128...",1.0,50.5,0.0,0.5
